In [17]:
import numpy as np
from LinAlgFunctions import *

ModuleNotFoundError: No module named 'LinAlgFunctions'

In [18]:
A = np.array([[ 0.5       , -0.09090909, -0.09090909, -0.04545455, -0.10606061],
       [-0.09090909,  0.18181818,  0.01515152,  0.        , -0.01515152],
       [-0.09090909,  0.01515152,  0.31818182,  0.09090909, -0.01515152],
       [-0.04545455,  0.        ,  0.09090909,  0.34090909, -0.09090909],
       [-0.10606061, -0.01515152, -0.01515152, -0.09090909,  0.34090909]])

# Create exact solution and vector b
x_exact = np.ones(5)
b = A@x_exact

# Create Identity matrix
I = np.eye(5)

In [19]:
# Compute Cholesky decomposition


def Cholesky(A):
    n = A.shape[0] # Get the number of rows (or columns) of the square matrix A
    L = np.zeros_like(A) # Initialize an empty lower triangular matrix L
    
    for k in range(n):
        # Compute the diagonal element of L at position (k, k)
        L[k, k] = np.sqrt(A[k, k] - np.dot(L[k, :k], L[k, :k]))
        for i in range(k + 1, n):
            L[i, k] = (A[i, k] - np.dot(L[i, :k], L[k, :k])) / L[k, k]
    
    return L  # Return the computed lower triangular matrix L
# Perform the Cholesky decomposition
# The function will decompose a given symmetric positive definite matrix A into L such that A = L @ L.T
L=Cholesky(A)
# Check if the Cholesky decomposition is correct by verifying ||L @ L.T - A|| (should be near zero)
# This step calculates the infinity norm of the difference between L @ L.T and A

# Check if the Cholesky decomposition it correct (norm should be near zero)
print(np.linalg.norm(L@L.T - A, np.inf))

1.3877787807814457e-16


In [20]:
# Compute inverse of matrix L

import numpy as np

def InvertL(L):
    n = L.shape[0] # Get the size of the square matrix L
    L_inv = np.zeros_like(L) # Initialize an empty matrix for the inverse

    for k in range(n):
        # Compute the diagonal element of the inverse (1 / L[k, k])
        L_inv[k, k] = 1. / L[k, k]
        for i in range(k + 1, n):
            # Compute the off-diagonal elements of the inverse
            # L_inv[i, k] = -(sum of L[i, k:i] * L_inv[k:i, k]) / L[i, i]
            L_inv[i, k] = -(L[i,k:i]@L_inv[k:i,k])/L[i,i]
    
   
    return L_inv
    # Compute the inverse of L using the InvertL function
Linv = InvertL(L)

# Check if the inversion is correct by verifying ||L @ Linv - I|| (should be near zero)
# This step calculates the infinity norm of the difference between L @ Linv and the identity matrix I
# Check if inversion is correct (norm should be near zero)
print(np.linalg.norm(L@Linv - I, np.inf))

9.860316562520647e-17


In [21]:
import numpy as np

def PLU(A):
   
    N = A.shape[0]  # Get the size of the square matrix A
    P = np.eye(N)  # Initialize the permutation matrix as an identity matrix
    L = np.eye(N)  # Initialize the lower triangular matrix as an identity matrix
    U = A.copy()   # Copy A to perform row operations without altering the original matrix

    for i in range(N):
        # Find the row with the maximum pivot element in the current column (partial pivoting)
        r_max = np.argmax(np.abs(U[i:, i])) + i

        # Swap rows in U and update P and L accordingly
        if r_max != i:
            U[[i, r_max], :] = U[[r_max, i], :]  # Swap rows in U
            P[[i, r_max], :] = P[[r_max, i], :]  # Swap rows in P
            if i > 0:
                L[[i, r_max], :i] = L[[r_max, i], :i]  # Adjust L for swapped rows

        # Eliminate entries below the pivot element
        for j in range(i + 1, N):
            factor = U[j, i] / U[i, i]  # Compute the elimination factor
            L[j, i] = factor            # Store the factor in L
            U[j, i:] -= factor * U[i, i:]  # Update the row in U to make entries below the pivot zero

    return P, L, U


def forward(L, b):
   
    y = np.zeros_like(b)  # Initialize solution vector y with zeros
    for i in range(len(b)):
        # Compute each element of y sequentially, using previously computed values
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
    return y


def backward(U, y):
   
    N = len(y)  # Get the size of the vector
    x = np.zeros_like(y)  # Initialize solution vector x with zeros
    for i in range(N-1, -1, -1):  # Work backwards from the last row to the first
        # Compute each element of x sequentially, using already computed values
        sum_Ux = np.dot(U[i, i+1:], x[i+1:])  # Sum up the contributions of already computed x values
        x[i] = (y[i] - sum_Ux) / U[i, i]  # Solve for the current variable
    return x


def PLUSolve(A, b):
    
    P, L, U = PLU(A)  # Perform PLU decomposition
    b_n = np.dot(P, b)  # Apply the permutation matrix to b
    y = forward(L, b_n)  # Solve Ly = b_n using forward substitution
    x = backward(U, y)  # Solve Ux = y using backward substitution
    return x


# Example usage:
x = PLUSolve(A, b)  # Solve Ax = b using PLUSolve

# Check the solution accuracy by comparing with the exact solution
print(np.linalg.norm(x - x_exact, np.inf))  # Norm should be near zero for an accurate solution


3.3306690738754696e-16


In [22]:
# Check vector 2-norm
def vector_2norm(x):
  return np.sqrt(np.sum(x**2))
print(np.linalg.norm(x_exact, 2) - vector_2norm(x_exact))

0.0


In [23]:
# Check matrix infinity-norm
def matrix_inf_norm(A):
    abs_A = np.abs(A)
    row_sums = np.sum(abs_A, axis=1) #sum each row 
    return np.max(row_sums) #return the max row after summation 
    
print(np.linalg.norm(A, np.inf) - matrix_inf_norm(A))

0.0


In [24]:
# Check inner product
def inner_product(x, y):
    return np.sum(x * y)  #< x, y >
print(np.inner(x_exact, b) - inner_product(x_exact, b))

0.0


In [25]:
# Check Neumann series computation
# Get the size of the matrix A (assuming square matrix)
def NeumannSeries(A, w=1):
    N = A.shape[0]
    D = np.diag(np.diag(A)) # D is the diagonal part of A
    L = np.tril(A, -1)# L is the strictly lower triangular part of A
    U = np.triu(A, 1)  # U is the strictly upper triangular part of A
    M = D + w * L # M is D + w * L, a combination of diagonal and lower triangular parts
    B = np.eye(N) - M / np.max(np.abs(M))
    # Use a Neumann series to approximate M^{-1}
    Minv = np.eye(N) # Minv starts as an identity matrix, which we will update iteratively
    while matrix_inf_norm(np.eye(N) - (Minv/np.max(M))@M) > 1e-12:
        Minv = np.eye(N) + B @ Minv
    Minv /= np.max(np.abs(M))
    return Minv
w = 1
B = NeumannSeries(A) # Call the NeumannSeries function to compute the inverse approximation
# Check that the computed inverse is correct by comparing with the exact inverse
# The norm should be close to zero if the approximation is good
# Check that inverse was computed correctly (norm should be near zero)
print(np.linalg.norm(B - np.linalg.inv(I - A), np.inf))

5.585295799541611


In [26]:
def Jacobi(A, b):
    # Dinv is the inverse of the diagonal elements of A (element-wise inverse)
    Dinv = 1 / np.diag(A)
    
    # L is the strictly lower triangular part of A
    L = np.tril(A, k=-1)
    
    # U is the strictly upper triangular part of A
    U = np.triu(A, k=1)
    
    # M is the combination of L and U (A = D + M)
    M = L + U
    
    # Initial guess for the solution (u) is a zero vector
    u = np.zeros_like(b)
    
    # Residual vector (r) is the difference between b and A @ u
    r = b - A @ u
    
    # Initialize the array to store the relative residuals for each iteration
    Jacobi = np.array([])
    
    # Iterate until the relative residual is sufficiently small
    while vector_2norm(r) / vector_2norm(b) > 1e-6:
        # Update the solution approximation u using Jacobi method formula
        u = Dinv * (b - M @ u)
        
        # Update the residual vector r
        r = b - A @ u
        
        # Store the relative residual in Jacobi for convergence monitoring
        Jacobi = np.append(Jacobi, vector_2norm(r) / vector_2norm(b))
    
    # Return the computed solution vector u
    return u

x = Jacobi(A, b)  # Solve for x using the Jacobi method

# Check the solution by comparing the computed solution to the exact solution
# The norm should be close to zero if the approximation is accurate
print(np.linalg.norm(x - x_exact, np.inf))

8.149701404835952e-07


In [27]:
def GaussSeidel(A, b):
    
    N = A.shape[0]  # Get the size of the matrix A
    
    # Extract the diagonal of A
    D = np.diag(np.diag(A))  
    
    # Extract the strictly lower triangular part of A
    L = np.tril(A, -1)
    
    # Compute Minv, an approximate inverse of M = D + L, using the Neumann Series
    Minv = NeumannSeries(A)
    
    # Extract the strictly upper triangular part of A
    U = np.triu(A, 1)
    
    # Define M as the sum of the diagonal and lower triangular parts
    M = D + L
    
    # Initialize the solution vector with zeros
    u = np.zeros((N,))
    
    # Compute the initial residual (difference between b and Ax)
    r = b - A @ u
    
    # Array to track the relative residuals for each iteration
    Gauss = np.array([])
    
    # Iterate until the relative residual is below the tolerance threshold
    while vector_2norm(r) / vector_2norm(b) > 1e-6:
        # Update the solution vector using the Gauss-Seidel formula
        u = Minv @ (b - U @ u)
        
        # Recompute the residual vector
        r = b - A @ u
        
        # Store the relative residual for convergence tracking
        Gauss = np.append(Gauss, vector_2norm(r) / vector_2norm(b))
    
    return u

# Example usage:
x = GaussSeidel(A, b)  # Solve Ax = b using Gauss-Seidel

# Check the solution accuracy by comparing with the exact solution
# The norm should be near zero for a correct solution
print(np.linalg.norm(x - x_exact, np.inf))


4.5870106868406424e-07


In [28]:

# Function to compute the infinity norm of a matrix
def matrix_inf_norm(A):
    return np.max(np.sum(np.abs(A), axis=1))

# Function to compute the 2-norm of a vector
def vector_2norm(x):
    return np.sqrt(np.sum(x**2))

# Define SOR function
def SOR(A, b, w):
    N = A.shape[0]
    D = np.diag(np.diag(A))
    L = np.tril(A, -1)
    U = np.triu(A, 1)
    M = D + w * L
    B = np.eye(N) - M / np.max(M)

    # Use a Neumann series to approximate M^{-1}
    Minv = np.eye(N)
    while matrix_inf_norm(np.eye(N) - (Minv / np.max(M)) @ M) > 1e-12:
        Minv = np.eye(N) + B @ Minv
    Minv /= np.max(M)
    n= A.shape[0]
    u = np.zeros(n)
    r = b - A @ u
    residuals = []

    # Implement SOR iterations
    while vector_2norm(r) / vector_2norm(b) > 1e-6:
        u = Minv @ (w * (b - U @ u) + (1. - w) * D @ u)
        r = b - A @ u
        residuals.append(vector_2norm(r) / vector_2norm(b))

    return u



# Set relaxation parameter w
w = 1.2  # Change this value as needed

# Solve Ax = b using SOR
x = SOR(A, b, w)


# Check the solution (norm should be near zero)
print(np.linalg.norm(x - x_exact, np.inf))


3.654470455138892e-07


In [29]:
# Solve Ax=b using conjugate gradient
def CG(A, b):
    N = A.shape[0]
    u = np.zeros((N,))
    r = b - A @ u
    delta = inner_product(r, r)
    bdelta = inner_product(b, b)
    p = r
    CG_residuals = np.array([])
    while delta > bdelta * (1e-6)**2:
        s = A @ p
        alpha = delta / inner_product(p, s)
        u = u + alpha * p
        r = r - alpha * s
        delta_new = inner_product(r, r)
        p = r + (delta_new / delta) * p
        delta = delta_new

        CG_residuals = np.append(CG_residuals, vector_2norm(r) / vector_2norm(b))

    return u

# Assuming A, b, and x_exact are defined
x = CG(A, b)

# Check solution (norm should be near zero)
print(np.linalg.norm(x - x_exact, np.inf))

3.3306690738754696e-16


In [30]:
# Solve Ax=b using preconditioned conjugate gradient. PCG will call incomplete Cholesky.


# Incomplete Cholesky Decomposition Function
def IncompleteCholesky(A):
    
    n = A.shape[0]
    L = np.zeros_like(A)

    for i in range(n):
        for j in range(i + 1):
            if i == j:
                # Compute the diagonal entries of L
                L[i, j] = np.sqrt(A[i, i] - np.sum(L[i, :j] ** 2))
            else:
                # Compute the off-diagonal entries of L
                L[i, j] = (A[i, j] - np.sum(L[i, :j] * L[j, :j])) / L[j, j]
    
    return L

# Preconditioned Conjugate Gradient (PCG) Method
def PCG(A, b, P=None):
   
    n = A.shape[0]
    x = np.zeros(n)  # Initial guess for x
    r = b - A @ x  # Initial residual
    if P is None:
        # Compute preconditioner using incomplete Cholesky decomposition
        L = IncompleteCholesky(A)
        P = np.linalg.inv(L @ L.T)  # Compute the approximate inverse of the preconditioner
    z = P @ r  # Apply the preconditioner
    p = z  # Initialize the conjugate direction
    rs_old = np.dot(r, z)  # Initial value of r^T z

    for _ in range(n):
        Ap = A @ p  # Matrix-vector product
        alpha = rs_old / np.dot(p, Ap)  # Compute step size
        x = x + alpha * p  # Update solution
        r = r - alpha * Ap  # Update residual
        
        # Check for convergence (norm of residual close to zero)
        if np.linalg.norm(r) < 1e-6:
            break
        
        z = P @ r  # Apply the preconditioner
        rs_new = np.dot(r, z)  # Compute new r^T z
        p = z + (rs_new / rs_old) * p  # Update conjugate direction
        rs_old = rs_new  # Update rs_old for the next iteration
    
    return x


# Solve Ax = b using PCG
x = PCG(A, b)



# Check solution (norm should be near zero)
print(np.linalg.norm(x - x_exact, np.inf))

4.440892098500626e-16
